In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import scipy.stats as sts
from datetime import datetime

df = pd.read_excel("BakeryData_Vilnius.xlsx")
stores = ["main street A","main street B","station A","station B"]
daysOfTheWeek = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df["weekday_text"] = df['date'].dt.strftime('%A')
df["date"] = pd.to_datetime(df["date"]).dt.date
df

,date,weekday,main street A,main street B,station A,station B,weekday_text
0,2016-05-11,3,2.23,NaN,NaN,NaN,Wednesday
1,2016-05-12,4,18.10,NaN,NaN,NaN,Thursday
2,2016-05-13,5,15.85,NaN,NaN,NaN,Friday
3,2016-05-14,6,14.22,NaN,NaN,NaN,Saturday
4,2016-05-15,7,2.58,NaN,NaN,NaN,Sunday
...,...,...,...,...,...,...,...
2572,2023-05-27,6,168.05,32.34,76.97,114.30,Saturday
2573,2023-05-28,7,44.62,32.85,80.21,91.25,Sunday
2574,2023-05-29,1,64.11,116.84,149.75,92.56,Monday
2575,2023-05-30,2,103.63,134.48,194.03,75.63,Tuesday


In [74]:
begin = datetime(day = 31,month = 12,year = 2016).date()
pre_covid = datetime(day = 1,month = 3,year = 2021).date()
after_covid = datetime(day = 1,month = 3,year = 2022).date()
df = df[((df["date"] < pre_covid) | (df["date"] > after_covid)) & (df["date"] > begin)]

def removeOutliers(df):
    mean = df.mean()
    sd = df.std()
    return df[(df > (mean - 3*sd)) & (df < (mean + 3*sd))]

In [84]:
class Store():
    def __init__(self, name, p, pl, c, cs, dist):
        self.name = name
        self.p = p
        self.pl = pl
        self.c = c
        self.cs = cs
        self.overageCost = self.p - self.c
        self.underageCost = self.c + self.cs - self.pl
        self.serviceLevel = self.overageCost / ( self.overageCost + self.underageCost)
        self.dist = dist
    def fit(self, feature):
        if self.dist == sts.norm:
            return np.mean(feature), np.std(feature, ddof=1)
        if self.dist == sts.lognorm:
            return np.std(np.log(feature),ddof=1),0,np.exp(np.mean(np.log(feature)))
        else:
            raise Exception("MLE not found for given dist")
    def feature(self, day):
        feature = df[df["weekday_text"] == day][self.name]
        feature = feature[~np.isnan(feature)]
        return removeOutliers(feature)
    def bootstrap(self, day, m, alpha):
        feature = self.feature(day)
        params = self.fit(feature)
        vQ_hat = np.zeros(m)
        n = len(feature)
        for i in range(m):
            generatedData = self.dist.rvs(*params, size=n)
            btParams = self.fit(generatedData)
            vQ_hat[i] = self.dist.ppf(self.serviceLevel, *btParams)
        return np.quantile(vQ_hat, [alpha/2, 1-alpha/2])
    def nonParametricOptimalInterval(self, day, alpha):
        feature = self.feature(day)
        criticalAmount = feature.quantile(self.serviceLevel)

        # Find interval
        z = 1 / sts.norm.ppf(1-alpha/2)
        feature = np.sort(feature)
        n = len(feature)
        deviation = (z * np.sqrt(n*self.serviceLevel * (1-self.serviceLevel)))
        upperBound = int(n*self.serviceLevel + deviation)
        lowerBound = int(n*self.serviceLevel - deviation)
        if upperBound > n:
            upperBound = n
        if lowerBound < 1:
            lowerBound = 1
        minimumAmount, maximumAmount  = feature[lowerBound - 1], feature[upperBound - 1]
        return {'store':self.name,'day':day,'method' : "nonParametric",'optimalQuantity': criticalAmount, 'lower bound': minimumAmount, 'upper bound': maximumAmount}
    def parametricOptimalInterval(self, day, alpha, m=100):
        feature = self.feature(day)
        params = self.fit(feature)
        optimalQuantity = self.dist.ppf(self.serviceLevel, *params)
        minimumAmount, maximumAmount = self.bootstrap(day,m,alpha)
        
        return {'store':self.name,'day':day,'method' : "Parametric",'optimalQuantity': optimalQuantity, 'lower bound': minimumAmount, 'upper bound': maximumAmount}

mainstreetA = Store("main street A", 4.64, 0.15, 3.85, 0.11,  sts.lognorm)
stationA = Store("station A", 4.64, 0.15, 4.16, 0.08, sts.norm)
#mainstreetB = Store("main street A", 4.64, 0.15, 3.42, 0.08)
#stationB = Store("station B", 4.64, 0.15, 3.32, 0.09)
Stores = [mainstreetA, stationA]

m = 5000
output = []
for store in Stores:
    for day in daysOfTheWeek:
        output.append(store.nonParametricOptimalInterval(day,0.05))
        output.append(store.parametricOptimalInterval(day,0.05,m))

outputdf = pd.DataFrame(output)
outputdf["length"] = np.abs(outputdf["lower bound"]-outputdf["upper bound"])
outputdf

,store,day,method,optimalQuantity,lower bound,upper bound,length
0,main street A,Monday,nonParametric,40.953217,40.540000,41.300000,0.760000
1,main street A,Monday,Parametric,40.305496,38.636225,42.132232,3.496007
2,main street A,Tuesday,nonParametric,40.690043,39.960000,40.820000,0.860000
3,main street A,Tuesday,Parametric,40.802241,39.125013,42.592253,3.467240
4,main street A,Wednesday,nonParametric,41.398304,40.280000,41.860000,1.580000
5,main street A,Wednesday,Parametric,40.796702,39.146399,42.585414,3.439014
6,main street A,Thursday,nonParametric,42.580000,42.010000,42.900000,0.890000
7,main street A,Thursday,Parametric,41.383261,39.731997,43.182689,3.450692
8,main street A,Friday,nonParametric,93.915761,93.690000,94.040000,0.350000
9,main street A,Friday,Parametric,93.983035,93.310087,94.659217,1.349130


In [ ]:
aggfuncList = {}

table = pd.pivot_table(df, values=stores, index='weekday_text', aggfunc=[np.mean, np.std])
table.columns = [f'{col}_{agg}' for col, agg in table.columns]
table



,mean_main street A,mean_main street B,mean_station A,mean_station B,std_main street A,std_main street B,std_station A,std_station B
weekday_text,,,,,,,,
Friday,98.412021,122.090000,153.224162,132.309362,6.109296,17.412300,29.017888,57.539247
Monday,56.584574,121.460556,152.209086,113.832766,17.425407,16.190518,27.752462,48.264783
Saturday,151.892021,35.314118,75.029848,103.749362,13.368080,5.308079,8.293490,11.284882
Sunday,58.234134,31.198824,75.152690,107.849787,19.130946,6.676195,8.083250,13.049768
Thursday,57.753156,122.245294,150.890609,115.332553,17.425922,14.300052,30.193218,42.141389
Tuesday,57.407234,128.156667,150.601574,120.033830,17.997409,19.212268,29.530480,46.692708
Wednesday,57.596537,130.295556,152.018990,135.590625,18.493259,11.115627,30.390583,60.714259


In [ ]:
df.describe()

,weekday,main street A,main street B,station A,station B
count,1976.000000,1976.00000,122.000000,1380.000000,330.000000
mean,4.001012,76.82080,99.367623,129.891326,118.437606
std,2.000759,37.44635,43.547492,42.839089,45.077501
min,1.000000,15.10000,18.700000,50.240000,42.510000
25%,2.000000,48.56500,40.442500,84.117500,90.660000
50%,4.000000,64.38500,118.115000,136.020000,108.890000
75%,6.000000,97.46000,131.645000,160.777500,129.307500
max,7.000000,191.88000,163.400000,275.720000,300.980000
